In [1]:
import torch
from torch import nn, optim
import import_ipynb
from model import net
from dataset import dataloader, val_loader
from tqdm import tqdm, tqdm_notebook

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from dataset.ipynb


100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:06<00:00, 289.62it/s]


In [2]:
# 分段的平台式衰減方法
epoch_lr = [
    (1000, 0.1),
    (100, 0.01),
    (100, 0.001),
    (100, 0.0001),
]

In [3]:
device = (
    torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)
#多標籤分類損失函數
criteron = nn.MultiLabelSoftMarginLoss()

In [4]:
def train():
    net.to(device)
    accuracies = []
    losses = []
    val_accuracies = []
    val_losses = []
    for n, (num_epoch, lr) in enumerate(epoch_lr):
        optimizer = optim.SGD(
            net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4
        )
        for epoch in range(num_epoch):
            # 每次驗證都是eval，所以訓練時要切換回train
            net.train()
            epoch_loss = 0.0
            epoch_acc = 0.0
            for i,(img, label) in tqdm(enumerate(dataloader)):
                out = net(img.to(device))
                label = label.to(device)
                optimizer.zero_grad()
                loss = criteron(out, label.to(device))
                loss.backward()
                optimizer.step()
                # 整理output
                predict = torch.argmax(out.view(-1, 36), dim=1)
                true_label = torch.argmax(label.view(-1, 36), dim=1)
                epoch_acc += torch.sum(predict == true_label).item()
                epoch_loss += loss.item()
                # 訓練3次驗證一次
            if epoch % 3 == 0:
                with torch.no_grad():
                    net.eval()
                    val_loss = 0.0
                    val_acc = 0.0
                    for i, (img, label) in tqdm_notebook(enumerate(val_loader)):
                        out = net(img.to(device))
                        label = label.to(device)
                        loss = criteron(out, label.to(device))
                        predict = torch.argmax(out.view(-1, 36), dim=1)
                        true_label = torch.argmax(label.view(-1, 36), dim=1)
                        val_acc += torch.sum(predict == true_label).item()
                        val_loss += loss.item()
                val_acc /= len(val_loader.dataset) * 4
                val_loss /= len(val_loader)
            epoch_acc /= len(dataloader.dataset) * 4
            epoch_loss /= len(dataloader)
            print("epoch : {}, epoch loss : {:.8f}, epoch accuracy : {:.8f}".format(
                epoch + sum([e[0] for e in epoch_lr[:n]]), epoch_loss, epoch_acc)
                 )
            if epoch % 3 == 0:
                print("epoch : {}, val loss: {:.8f}, val accuracy : {:.8f}".format(
                    epoch + sum([e[0] for e in epoch_lr[:n]]), val_loss, val_acc)
                     )
                for i in range(3):
                    val_accuracies.append(val_acc)
                    val_losses.append(val_loss)
                
            accuracies.append(epoch_acc)
            losses.append(epoch_loss)
            torch.save(
                            net.state_dict(),
                            './checkpoint/temp.pth',
                        )

In [5]:
if __name__ == '__main__':
    train()

0it [00:00, ?it/s]

torch.Size([128, 64, 15, 40])


0it [00:02, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x38400 and 8960x1024)